In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.tri as MTri

In [2]:
# Optimization Parameters
ConstitutiveModels = ['Ogden', 'NeoHookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Real']
BCsType  = BCsTypes[1] + ' Boundary Conditions'


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Optim_Path = os.path.join('../Optimization/Results/Compressible', BCsType, ConstitutiveModel)

OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv('Data/Grid.csv', sep=',', decimal='.', index_col=0)
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')
MarcData = pd.read_csv(os.path.join(Optim_Path, MarcName), sep=' ', decimal='.')

# Add Lambda
OptiData['Lambda'] = 2*OptiData['Mu']*OptiData['Nu']/(1-2*OptiData['Nu'])
MarcData['Lambda'] = 2*MarcData['Mu']*MarcData['Nu']/(1-2*MarcData['Nu'])

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCostNormMax'].idxmin()])

Nus     = CostData.Nu
Lambdas = CostData.Lambda
Mus     = CostData.Mu
Alphas  = CostData.Alpha
Costs   = CostData.TotalCostNormMax

DM, MuM, AlphaM = 1.026, 0.458, -59.367
NuM = (3-DM*MuM)/(DM*MuM+6)
Marcdf = pd.DataFrame([NuM, MuM, AlphaM])

Subsampling = 1

# # 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c='Black', marker = '.', label='Points tested')
# CostImage = ax.scatter(Lambdas[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c='Black', marker = '.', label='Points tested')
ax.set_xlabel('Nu (-)')
# ax.set_xlabel('Lambda (kPa)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.legend()
plt.show()

Min cost point for:
Alpha                             -24
BCsType                         Fixed
CompressionCostNoNorm       0.0296841
CompressionCostNormMax      0.0531391
CompressionCostStep           9.18372
ConstitutiveModel               Ogden
Mu                                0.6
Nu                               0.41
NumberElements                     20
SimpleShearCostNoNorm      0.00813879
SimpleShearCostNormMax       0.116729
SimpleShearCostStep            2.2651
TensileCostNoNorm         0.000807557
TensileCostNormMax           0.032357
TensileCostStep               1196.05
TotalCostNoNorm             0.0128768
TotalCostNormMax            0.0674084
TotalCostStep                   402.5
Lambda                        2.73333
Name: 41218, dtype: object


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# Create interpolated data and the lambda dimension

Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color = Colors[0]

Xarray = np.sort(CostData['Nu'].unique())
Yarray = np.sort(CostData['Mu'].unique())
Zarray = np.sort(CostData['Alpha'].unique())

GridPoint = np.meshgrid(Xarray,Yarray,Zarray)

Xarray = GridPoint[0]
Xarray = np.reshape(Xarray,Xarray.shape[0]*Xarray.shape[1]*Xarray.shape[2])
Yarray = GridPoint[1]
Yarray = np.reshape(Yarray,Yarray.shape[0]*Yarray.shape[1]*Yarray.shape[2])
Zarray = GridPoint[2]
Zarray = np.reshape(Zarray,Zarray.shape[0]*Zarray.shape[1]*Zarray.shape[2])
CostArray = np.ones(len(Xarray))*np.nan

InterpData = pd.DataFrame({'Nu':Xarray,'Mu':Yarray,'Alpha':Zarray,'Cost':CostArray})

for Index in range(len(Xarray)):
    Filter1 = CostData.Nu == Xarray[Index]
    Filter2 = CostData.Mu == Yarray[Index]
    Filter3 = CostData.Alpha == Zarray[Index]

    if CostData[Filter1&Filter2&Filter3][Color].empty:
        InterpData['Cost'].loc[Index] = np.nan
    else:
        InterpData['Cost'].loc[Index] = CostData[Filter1&Filter2&Filter3][Color].values

InterpData['Cost'] = InterpData['Cost'].interpolate()

InterpData['Lambda'] = 2*InterpData['Mu']*InterpData['Nu']/(1-2*InterpData['Nu'])
InterpData.to_csv('Data/InterpGrid.csv')

In [4]:
# Plot

Subsampling = 1

Nus     = InterpData.Nu
Mus     = InterpData.Mu
Alphas  = InterpData.Alpha
Lambdas = InterpData.Lambda
Costs   = InterpData.Cost

%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs, marker = '.', label='Points tested')
CostImage = ax.scatter(Lambdas[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs, marker = '.', label='Points tested')
Colorbar = plt.colorbar(CostImage)
Colorbar.set_label('Cost')
# ax.set_xlabel('Nu (-)')
ax.set_xlabel('Lambdas (kPa)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Filtered 3D Plot

FilteredRangeValues = [0,0.1]

Filter1 = InterpData['Cost'] > FilteredRangeValues[0]
Filter2 = InterpData['Cost'] < FilteredRangeValues[1]

FilteredCost = InterpData[Filter1 & Filter2]

%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus     = FilteredCost.Nu
Lambdas = FilteredCost.Lambda
Mus     = FilteredCost.Mu
Alphas  = FilteredCost.Alpha
Costs   = FilteredCost['Cost']

# img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_xlabel('Lambda (kPa)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[FilteredCost['Cost'].idxmin()]], [Mus[FilteredCost['Cost'].idxmin()]], [Alphas[FilteredCost['Cost'].idxmin()]], marker = 'o', color='k')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r')
ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m')
ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m')
ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c')
ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c')
ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b')
ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g')
# Build the legend
ax.plot([],[], marker = 'o', linestyle='none', color='m', label='Budday starting point')
ax.plot([],[], color='m', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='r', label='Optimization result')
ax.plot([],[], marker = 'o', linestyle='none', color='c', label='Marc starting point')
ax.plot([],[], color='c', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='b', label='Optimization result')
ax.plot([],[], marker = 'o', linestyle='none', color='g', label='Marc optimization result')
ax.plot([],[], marker = '.', linestyle='none', color='y', label='Points tested')
ax.plot([],[], marker = 'o', linestyle='none', color='k', label='Grid min cost point')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [250]:
MinPoint = InterpData['Cost'].idxmin()

AxisNames = InterpData.columns[[0,1,2]]
Units = [' (-)', ' (kPa)', ' (-)']

%matplotlib widget
Figure, Axes  = plt.subplots(1,3, figsize=(12,4))

for Projection in range(len(AxisNames)):
    
    # Filter data to get the correct slice
    SliceAxis = InterpData[AxisNames[Projection]].loc[MinPoint]
    Filter = InterpData[AxisNames[Projection]] == SliceAxis
    Slice = InterpData[Filter]
    
    # Data for plot
    XAxis = AxisNames[Projection-2]
    YAxis = AxisNames[Projection-1]
    
    XData = Slice[XAxis]
    YData = Slice[YAxis]
    Color = Slice['Cost'].values
    
    Table = Slice.pivot(XAxis,YAxis,'Cost')
    XPositions = Table.index.values
    YPositions = Table.columns.values
    Contourlevels = Table.transpose().values
    
    # Levels for contour
    SortedSlice = Slice.sort_values('Cost')
    SampledLevels = SortedSlice['Cost'][0::100].values
    Vmax = SampledLevels[-1]
    SampledLevels = np.append(SampledLevels,Slice['Cost'].max())
    
    # Plot
    Axes[Projection].scatter(XData,YData,c=Color,marker='.',s=10,vmax=Vmax)
    Axes[Projection].contour(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax)
    CostContour = Axes[Projection].contourf(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax,alpha=0.85)
    
    Axes[Projection].plot(InterpData[XAxis][MinPoint], InterpData[YAxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    Axes[Projection].plot(OptiData[XAxis][len(OptiData)-1], OptiData[YAxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    
    
    # Plot Settings
    Axes[Projection].set_xlabel(XAxis+Units[Projection-2])
    Axes[Projection].set_ylabel(YAxis+Units[Projection-1])
    
    ColorBar = Figure.colorbar(CostContour,ax=Axes[Projection])
    ColorBar.set_label('Cost (-)')
    
    if Projection+1 == len(AxisNames):
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
        plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(InterpData['Cost'].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
        Figure.legend(loc='lower center', framealpha=1, ncol=3)
        plt.subplots_adjust(left=0.075, bottom=0.25, right=0.95, top=0.95, wspace=0.4, hspace=None)
        
plt.savefig('Plots/MinPointProjections.pdf')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [266]:
Slice['Mu'].min()

0.1

In [215]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y, Z = axes3d.get_test_data(0.05)

# Plot the 3D surface
ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)

# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph
cset = ax.contour(X, Y, Z, zdir='z', offset=-100, cmap=cm.coolwarm)
cset = ax.contour(X, Y, Z, zdir='x', offset=-40, cmap=cm.coolwarm)
cset = ax.contour(X, Y, Z, zdir='y', offset=40, cmap=cm.coolwarm)

ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_zlim(-100, 100)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [254]:
X,Y = np.meshgrid(XPositions,YPositions)


Figure, Axis = plt.subplots(1, 1, figsize=(10, 7))
Axis.remove()
Axis = Figure.add_subplot(1,1,1, projection='3d')
Axis.contour(X,Y,Contourlevels,zdir='z',offset=0,levels=SampledLevels,vmax=Vmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [278]:
%matplotlib widget

Figure, Axis = plt.subplots(1, 1, figsize=(10, 7))
Axis.remove()
Axis = Figure.add_subplot(1,1,1, projection='3d')

AxisNames = InterpData.columns[[0,1,2]]
Units = [' (-)', ' (kPa)', ' (-)']
SliceAxes = ['X','Y','Z']

def f(x, y, z):
    
    InterpData = pd.read_csv('Data/InterpGrid.csv', index_col=0)
    
    AxisNames = InterpData.columns[[0,1,2]]
    XAxis = AxisNames[0]
    YAxis = AxisNames[1]
    ZAxis = AxisNames[2]
        
    # Filters
    Filter1 = InterpData[XAxis] == x
    Filter2 = InterpData[YAxis] == y
    Filter3 = InterpData[ZAxis] == z
        
    Filters = [Filter1,Filter2,Filter3]
    
    Point = InterpData[Filter1&Filter2&Filter3]
    
    Axis.clear()
    Axis.plot([x],[y],[z], marker='o')
    
    for Projection in range(len(Filters)):
        Projection = 2
        
        Slice = InterpData[Filters[Projection]]

        # Data for plot
        Axis1 = AxisNames[Projection-2]
        Axis2 = AxisNames[Projection-1]
        Axis3 = AxisNames[Projection]

        Data1 = Slice[Axis1]
        Data2 = Slice[Axis2]
        Color = Slice['Cost'].values

        Table = Slice.pivot(Axis1,Axis2,'Cost')
        Positions1 = Table.index.values
        Positions2 = Table.columns.values
        Contourlevels = Table.transpose().values

        Positions1, Positions2 = np.meshgrid(Positions1,Positions2)

        # Levels for contour
        SortedSlice = Slice.sort_values('Cost')
        SampledLevels = SortedSlice['Cost'][0::100].values
        Vmax = SampledLevels[-1]
        SampledLevels = np.append(SampledLevels,Slice['Cost'].max())

        # Plot contour
        Axis.contour(Positions1,Positions2,Contourlevels,zdir=SliceAxes[Projection],offset=InterpData[Axis3].min(),levels=SampledLevels,vmax=Vmax)

    # Plot settings
    Axis.set_xlim(InterpData[XAxis].min(), InterpData[XAxis].max())
    Axis.set_ylim(InterpData[YAxis].min(), InterpData[YAxis].max())
    Axis.set_zlim(InterpData[ZAxis].min(), InterpData[ZAxis].max())
    Axis.set_xlabel(XAxis)
    Axis.set_ylabel(YAxis)
    Axis.set_zlabel(ZAxis)
    
    # make the panes transparent
    Axis.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    Axis.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    Axis.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    # make the grid lines transparent
    Axis.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    Axis.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    Axis.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    
    plt.show()


Xmin, Xmax = CostData['Nu'].min(), CostData['Nu'].max()
Xstep = round((Xmax-Xmin)/(len(CostData['Nu'].unique())-1),3)
Ymin, Ymax = CostData['Mu'].min(), CostData['Mu'].max()
Ystep = round((Ymax-Ymin)/(len(CostData['Mu'].unique())-1),2)
Zmin, Zmax = CostData['Alpha'].min(), CostData['Alpha'].max()
Zstep = round((Zmax-Zmin)/(len(CostData['Alpha'].unique())-1),0)

    
interactive_plot = interactive(f, x=(Xmin, Xmax, Xstep), y=(Ymin, Ymax, Ystep), z=(Zmin, Zmax, Zstep))
interactive_plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.42, description='x', max=0.495, min=0.35, step=0.005), FloatSlider(v…

In [145]:
# Plotting
%matplotlib widget

# Plot data
Axes   = ['Nu', 'Mu', 'Alpha']
Units  = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color = Colors[0]
VMin  = 0
VMax  = 1
Transparency = 0.5

# FilteredCost = CostData.dropna()
# for Point in FilteredCost[Color].index:
#     CostValue = FilteredCost[Color][Point]
#     if CostValue < VMin or CostValue > VMax :
#         FilteredCost = FilteredCost.drop([Point])

MinPoint = CostData[Color].idxmin()
    
# for Projection in range(len(Axes)):
Projection = 0

# Plot tuning
Xaxis = 'Nu'
Yaxis = 'Mu'
Zaxis = 'Alpha'

# XY plane
Slice4Contour = [CostData[Zaxis][MinPoint]]
Slice = CostData.loc[CostData[Zaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Yaxis,Xaxis])
Slice = Slice.dropna()
XY_X = Slice[Xaxis].to_numpy()
XY_Y = Slice[Yaxis].to_numpy()
XY_Z = Slice[Zaxis].to_numpy()
XY_Colors = Slice[Color].to_numpy()

XY_Triangles = MTri.Triangulation(XY_X, XY_Y).triangles
XY_MeanColors = np.mean(XY_Colors[XY_Triangles], axis=1)
XY_Triangles = MTri.Triangulation(XY_X, XY_Y, XY_Triangles)

# YZ plane
Slice4Contour = [CostData[Xaxis][MinPoint]]
Slice = CostData.loc[CostData[Xaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Zaxis,Yaxis])
Slice = Slice.dropna()
YZ_X = Slice[Xaxis].to_numpy()
YZ_Y = Slice[Yaxis].to_numpy()
YZ_Z = Slice[Zaxis].to_numpy()
YZ_Colors = Slice[Color].to_numpy()

YZ_Triangles = MTri.Triangulation(YZ_Y, YZ_Z).triangles
YZ_MeanColors = np.mean(YZ_Colors[YZ_Triangles], axis=1)
YZ_Triangles = MTri.Triangulation(YZ_X, YZ_Y, YZ_Triangles)

# ZX plane
Slice4Contour = [CostData[Yaxis][MinPoint]]
Slice = CostData.loc[CostData[Yaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Xaxis,Zaxis])
Slice = Slice.dropna()
ZX_X = Slice[Xaxis].to_numpy()
ZX_Y = Slice[Yaxis].to_numpy()
ZX_Z = Slice[Zaxis].to_numpy()
ZX_Colors = Slice[Color].to_numpy()

ZX_Triangles = MTri.Triangulation(ZX_Z, ZX_X).triangles
ZX_MeanColors = np.mean(ZX_Colors[ZX_Triangles], axis=1)
ZX_Triangles = MTri.Triangulation(ZX_X, ZX_Y, ZX_Triangles)


plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.gca(projection='3d')
cmap = plt.get_cmap('viridis')
# CostImage = ax.scatter(FilteredCost[Xaxis], FilteredCost[Yaxis], FilteredCost[Zaxis], c=FilteredCost[Color], cmap=cmap, marker = '.', vmin=VMin, vmax=VMax)
XY_Surf = ax.plot_trisurf(XY_Triangles, XY_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
YZ_Surf = ax.plot_trisurf(YZ_Triangles, YZ_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
ZX_Surf = ax.plot_trisurf(ZX_Triangles, ZX_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
ax.plot([CostData[Xaxis][MinPoint]], [CostData[Yaxis][MinPoint]], [CostData[Zaxis][MinPoint]], marker = 'o', fillstyle='full', linestyle='none', color='k')
# ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m')
# ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m')
ax.plot([OptiData['Nu'].iloc[-1]], [OptiData['Mu'].iloc[-1]], [OptiData['Alpha'].iloc[-1]], marker = 'o', color='r')
# Build the legend
# ax.plot([],[], marker = 'o', linestyle='none', color='m', label='Budday starting point')
# ax.plot([],[], color='m', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='r', label='Optimization result')
ax.plot([],[], marker = 'o', linestyle='none', color='k', label='Grid min cost point')
plt.legend(loc='lower left', framealpha=1)
XY_Surf.set_array(XY_MeanColors)
YZ_Surf.set_array(YZ_MeanColors)
ZX_Surf.set_array(ZX_MeanColors)
ColorBar=plt.colorbar(ZX_Surf)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.set_xlim([CostData['Nu'].min(), CostData['Nu'].max()])
ax.set_ylim([CostData['Mu'].min(), CostData['Mu'].max()])
ax.set_zlim([CostData['Alpha'].min(), CostData['Alpha'].max()])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
%matplotlib widget

# Plot data
Axes  = ['Nu', 'Mu', 'Alpha']
Units = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[12,4])
fig = plt.figure()

MinPoint = CostData[Color].idxmin()
    
for Projection in range(len(Axes)):

    # Plot tuning
    Xaxis     = Axes[Indices[Projection]]
    Xunits    = Units[Indices[Projection]]
    Yaxis     = Axes[Indices[Projection+1]]
    Yunits    = Units[Indices[Projection+1]]
    SliceAxis = Axes[Indices[Projection+2]]

    # Generate data for contour
    Slice4Contour = [CostData[SliceAxis][MinPoint]]
    Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color]]
    Slice = Slice.drop_duplicates([Yaxis,Xaxis])

    Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

    i     = 1
    Pos   = int(i*len(Levels)/(NContours+1))
    ContourSelection = np.zeros(len(Levels))

    while Pos < len(Levels):
        ContourSelection[Pos] = 1
        i  += 1
        Pos = int(i*len(Levels)/(NContours+1))


    SampledLevels = ContourSelection*Levels
    SampledLevels = SampledLevels[SampledLevels != 0]
    
    SampledLevels = np.insert(SampledLevels, 0, 0)
    SampledLevels = np.append(SampledLevels,Levels[-1])

    Table = Slice.pivot(Yaxis,Xaxis,Color)
    Table = Table.dropna()

    X = Table.columns.values
    Y = Table.index.values
    Z = Table.values
    Z = Z.reshape(Z.size)

    XY,YX = np.meshgrid(X, Y)

    XY = XY.reshape(1,XY.size)[0]
    YX = YX.reshape(1,YX.size)[0]
    
    # Plot
    ax = fig.add_subplot(1,3,Projection+1)
    ax.scatter(XY, YX, c=Z, marker = '.', s=10, cmap='viridis', vmin=VMin, vmax=VMax)
    ax.tricontour(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, colors='black', linewidths=0.5)
    CostContour = ax.tricontourf(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, cmap='viridis', alpha=0.85)
    ax.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    ax.plot([OptiData[Xaxis].iloc[-1]], [OptiData[Yaxis].iloc[-1]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    if Projection+1 == 3:
        ColorBar=fig.colorbar(CostContour)
        ColorBar.set_label('Cost (-)')
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
        plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
        fig.legend(loc='lower center', framealpha=1, ncol=3)
    ax.set_xlabel(Xaxis + Xunits)
    ax.set_ylabel(Yaxis + Yunits)
    plt.subplots_adjust(left=0.05, bottom=0.25, right=0.9, top=0.95, wspace=0.3, hspace=None)
#     plt.savefig('CostFunctionGrid/MinPoint'+Color+'Results.pdf')
    plt.show()

In [ ]:
%matplotlib widget

# Plot data
Axes   = ['Nu', 'Mu', 'Alpha']
Units  = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[12,4])
fig = plt.figure()

MarcMinPoint = MarcData['TotalCost'].idxmin()
MarcNu = np.round(MarcData['Nu'].loc[MarcMinPoint]*2,1)/2
MarcMu = np.round(MarcData['Mu'].loc[MarcMinPoint],1)
MarcAlpha = np.round(MarcData['Alpha'].loc[MarcMinPoint],0)

Filter1 = CostData["Nu"] == MarcNu
Filter2 = CostData["Mu"] == MarcMu
Filter3 = CostData["Alpha"] == MarcAlpha
  
# filtering data on basis of both filters 
MinPoint = CostData[Filter1 & Filter2 & Filter3].index[0]

    
for Projection in range(len(Axes)):

    # Plot tuning
    Xaxis     = Axes[Indices[Projection]]
    Xunits    = Units[Indices[Projection]]
    Yaxis     = Axes[Indices[Projection+1]]
    Yunits    = Units[Indices[Projection+1]]
    SliceAxis = Axes[Indices[Projection+2]]

    # Generate data for contour
    Slice4Contour = [CostData[SliceAxis][MinPoint]]
    Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color]]
    Slice = Slice.drop_duplicates([Yaxis,Xaxis])

    Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

    i     = 1
    Pos   = int(i*len(Levels)/(NContours+1))
    ContourSelection = np.zeros(len(Levels))

    while Pos < len(Levels):
        ContourSelection[Pos] = 1
        i  += 1
        Pos = int(i*len(Levels)/(NContours+1))


    SampledLevels = ContourSelection*Levels
    SampledLevels = SampledLevels[SampledLevels != 0]
    
    SampledLevels = np.insert(SampledLevels, 0, 0)
    SampledLevels = np.append(SampledLevels,Levels[-1])

    Table = Slice.pivot(Yaxis,Xaxis,Color)
    Table = Table.dropna()

    X = Table.columns.values
    Y = Table.index.values
    Z = Table.values
    Z = Z.reshape(Z.size)

    XY,YX = np.meshgrid(X, Y)

    XY = XY.reshape(1,XY.size)[0]
    YX = YX.reshape(1,YX.size)[0]

    Weights = np.linspace(len(Levels)-1,1,len(Levels)-1)
    Slopes = []
    for Index in range(len(Levels)-1):
        Slope = Levels[Index+1]-Levels[Index]
        Slopes.append(Slope*Weights[Index])

    NewColorScale = np.linspace(1,len(Levels),len(Levels)) * np.mean(Slopes) / len(Levels)

    Vmin = 0
    Vmax = max(NewColorScale)
    
    # Plot
    ax = fig.add_subplot(1,3,Projection+1)
    ax.scatter(XY, YX, c=Z, marker = '.', s=10, cmap='viridis', vmin=Vmin, vmax=Vmax)
    ax.tricontour(XY, YX, Z, vmin=Vmin, vmax=Vmax, levels=SampledLevels, colors='black', linewidths=0.5)
    CostContour = ax.tricontourf(XY, YX, Z, vmin=Vmin, vmax=Vmax, levels=SampledLevels, cmap='viridis', alpha=0.85)
#     plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
#     plt.plot(OptiData[Xaxis][len(OptiData)-1], OptiData[Yaxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    ax.plot([MarcData[Xaxis][len(MarcData)-1]], [MarcData[Yaxis][len(MarcData)-1]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    if Projection+1 == 3:
        ColorBar=fig.colorbar(CostContour)
        ColorBar.set_label('Cost (-)')
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
#         plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
        fig.legend(loc='lower center', framealpha=1, ncol=3)
    ax.set_xlabel(Xaxis + Xunits)
    ax.set_ylabel(Yaxis + Yunits)
    plt.subplots_adjust(left=0.05, bottom=0.25, right=0.9, top=0.95, wspace=0.3, hspace=None)
#     plt.savefig('CostFunctionGrid/MarcPoint'+Color+'Results.pdf')
    plt.show()

In [ ]:
# Plotting
%matplotlib widget

# Plot data
Axes   = ['Nu', 'Mu', 'Alpha']
Units  = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color = Colors[0]
VMin  = 0
VMax  = 1
Transparency = 0.5


# FilteredCost = CostData.dropna()
# for Point in FilteredCost[Color].index:
#     CostValue = FilteredCost[Color][Point]
#     if CostValue < VMin or CostValue > VMax :
#         FilteredCost = FilteredCost.drop([Point])

MarcNu = np.round(MarcData['Nu'].iloc[-2]*2,2)/2
MarcMu = np.round(MarcData['Mu'].iloc[-2],1)
MarcAlpha = np.round(MarcData['Alpha'].iloc[-2],0)

Filter1 = CostData['Nu'] == MarcNu
Filter2 = CostData['Mu'] == MarcMu
Filter3 = CostData['Alpha'] == MarcAlpha
  
# filtering data on basis of both filters 
MinPoint = CostData[Filter1 & Filter2 & Filter3].index[0]
    
# for Projection in range(len(Axes)):
Projection = 0

# Plot tuning
Xaxis = 'Nu'
Yaxis = 'Mu'
Zaxis = 'Alpha'

# XY plane
Slice4Contour = [CostData[Zaxis][MinPoint]]
Slice = CostData.loc[CostData[Zaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Yaxis,Xaxis])
Slice = Slice.dropna()
XY_X = Slice[Xaxis].to_numpy()
XY_Y = Slice[Yaxis].to_numpy()
XY_Z = Slice[Zaxis].to_numpy()
XY_Colors = Slice[Color].to_numpy()

XY_Triangles = MTri.Triangulation(XY_X, XY_Y).triangles
XY_MeanColors = np.mean(XY_Colors[XY_Triangles], axis=1)
XY_Triangles = MTri.Triangulation(XY_X, XY_Y, XY_Triangles)

# YZ plane
Slice4Contour = [CostData[Xaxis][MinPoint]]
Slice = CostData.loc[CostData[Xaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Zaxis,Yaxis])
Slice = Slice.dropna()
YZ_X = Slice[Xaxis].to_numpy()
YZ_Y = Slice[Yaxis].to_numpy()
YZ_Z = Slice[Zaxis].to_numpy()
YZ_Colors = Slice[Color].to_numpy()

YZ_Triangles = MTri.Triangulation(YZ_Y, YZ_Z).triangles
YZ_MeanColors = np.mean(YZ_Colors[YZ_Triangles], axis=1)
YZ_Triangles = MTri.Triangulation(YZ_X, YZ_Y, YZ_Triangles)

# ZX plane
Slice4Contour = [CostData[Yaxis][MinPoint]]
Slice = CostData.loc[CostData[Yaxis].isin(Slice4Contour)][[Zaxis,Yaxis,Xaxis,Color]]
Slice = Slice.drop_duplicates([Xaxis,Zaxis])
Slice = Slice.dropna()
ZX_X = Slice[Xaxis].to_numpy()
ZX_Y = Slice[Yaxis].to_numpy()
ZX_Z = Slice[Zaxis].to_numpy()
ZX_Colors = Slice[Color].to_numpy()

ZX_Triangles = MTri.Triangulation(ZX_Z, ZX_X).triangles
ZX_MeanColors = np.mean(ZX_Colors[ZX_Triangles], axis=1)
ZX_Triangles = MTri.Triangulation(ZX_X, ZX_Y, ZX_Triangles)


plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.gca(projection='3d')
cmap = plt.get_cmap('viridis')
# CostImage = ax.scatter(FilteredCost[Xaxis], FilteredCost[Yaxis], FilteredCost[Zaxis], c=FilteredCost[Color], cmap=cmap, marker = '.', vmin=VMin, vmax=VMax)
XY_Surf = ax.plot_trisurf(XY_Triangles, XY_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
YZ_Surf = ax.plot_trisurf(YZ_Triangles, YZ_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
ZX_Surf = ax.plot_trisurf(ZX_Triangles, ZX_Z, cmap=cmap, shade=False, linewidth=0., alpha=Transparency, vmin=VMin, vmax=VMax)
ax.plot([MarcData['Nu'].iloc[-2]],[MarcData['Mu'].iloc[-2]], [MarcData['Alpha'].iloc[-2]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization final point')
plt.legend(loc='lower left', framealpha=1)
XY_Surf.set_array(XY_MeanColors)
YZ_Surf.set_array(YZ_MeanColors)
ZX_Surf.set_array(ZX_MeanColors)
ColorBar=plt.colorbar(ZX_Surf)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.set_xlim([CostData['Nu'].min(), CostData['Nu'].max()])
ax.set_ylim([CostData['Mu'].min(), CostData['Mu'].max()])
ax.set_zlim([CostData['Alpha'].min(), CostData['Alpha'].max()])
plt.show()

In [ ]:
%matplotlib widget

# Plot data
Axes   = ['Nu', 'Mu', 'Alpha']
Units  = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[12,4])
fig = plt.figure()

MarcMinPoint = MarcData['TotalCost'].idxmin()
MarcNu = np.round(MarcData['Nu'].loc[MarcMinPoint]*2,1)/2
MarcMu = np.round(MarcData['Mu'].loc[MarcMinPoint],1)
MarcAlpha = np.round(MarcData['Alpha'].loc[MarcMinPoint],0)

Filter1 = CostData["Nu"] == MarcNu
Filter2 = CostData["Mu"] == MarcMu
Filter3 = CostData["Alpha"] == MarcAlpha
  
# filtering data on basis of both filters 
MinPoint = CostData[Filter1 & Filter2 & Filter3].index[0]

    
for Projection in range(len(Axes)):

    # Plot tuning
    Xaxis     = Axes[Indices[Projection]]
    Xunits    = Units[Indices[Projection]]
    Yaxis     = Axes[Indices[Projection+1]]
    Yunits    = Units[Indices[Projection+1]]
    SliceAxis = Axes[Indices[Projection+2]]

    # Generate data for contour
    Slice4Contour = [CostData[SliceAxis][MinPoint]]
    Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color]]
    Slice = Slice.drop_duplicates([Yaxis,Xaxis])

    Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

    i     = 1
    Pos   = int(i*len(Levels)/(NContours+1))
    ContourSelection = np.zeros(len(Levels))

    while Pos < len(Levels):
        ContourSelection[Pos] = 1
        i  += 1
        Pos = int(i*len(Levels)/(NContours+1))


    SampledLevels = ContourSelection*Levels
    SampledLevels = SampledLevels[SampledLevels != 0]
    
    SampledLevels = np.insert(SampledLevels, 0, 0)
    SampledLevels = np.append(SampledLevels,Levels[-1])

    Table = Slice.pivot(Yaxis,Xaxis,Color)
    Table = Table.dropna()

    X = Table.columns.values
    Y = Table.index.values
    Z = Table.values
    Z = Z.reshape(Z.size)

    XY,YX = np.meshgrid(X, Y)

    XY = XY.reshape(1,XY.size)[0]
    YX = YX.reshape(1,YX.size)[0]

    # Plot
    ax = fig.add_subplot(1,3,Projection+1)
    ax.scatter(XY, YX, c=Z, marker = '.', s=10, cmap='viridis', vmin=VMin, vmax=VMax)
    ax.tricontour(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, colors='black', linewidths=0.5)
    CostContour = ax.tricontourf(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, cmap='viridis', alpha=0.85)
#     plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
#     plt.plot(OptiData[Xaxis][len(OptiData)-1], OptiData[Yaxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    ax.plot([MarcData[Xaxis][len(MarcData)-1]], [MarcData[Yaxis][len(MarcData)-1]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    if Projection+1 == 3:
        ColorBar=fig.colorbar(CostContour)
        ColorBar.set_label('Cost (-)')
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
#         plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
        fig.legend(loc='lower center', framealpha=1, ncol=3)
    ax.set_xlabel(Xaxis + Xunits)
    ax.set_ylabel(Yaxis + Yunits)
    plt.subplots_adjust(left=0.05, bottom=0.25, right=0.9, top=0.95, wspace=0.3, hspace=None)
#     plt.savefig('CostFunctionGrid/MarcPoint'+Color+'Results.pdf')
    plt.show()

In [ ]:
%matplotlib widget

# Plot data
Axes   = ['Nu', 'Mu', 'Alpha']
Units  = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[12,4])
fig = plt.figure()

MarcMinPoint = MarcData['TotalCost'].idxmin()
MarcNu = np.round(MarcData['Nu'].loc[MarcMinPoint]*2,1)/2
MarcMu = np.round(MarcData['Mu'].loc[MarcMinPoint],1)
MarcAlpha = np.round(MarcData['Alpha'].loc[MarcMinPoint],0)

Filter1 = CostData["Nu"] == MarcNu
Filter2 = CostData["Mu"] == MarcMu
Filter3 = CostData["Alpha"] == MarcAlpha
  
# filtering data on basis of both filters 
MinPoint = CostData[Filter1 & Filter2 & Filter3].index[0]

def f(CompressionWeight, TensileWeight, SimpleShearWeight):
    plt.clf()
    
    for Projection in range(len(Axes)):

        # Plot tuning
        Xaxis     = Axes[Indices[Projection]]
        Xunits    = Units[Indices[Projection]]
        Yaxis     = Axes[Indices[Projection+1]]
        Yunits    = Units[Indices[Projection+1]]
        SliceAxis = Axes[Indices[Projection+2]]

        # Generate data for contour
        Slice4Contour = [CostData[SliceAxis][MinPoint]]
        Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color,Colors[4],Colors[7],Colors[10]]]
        Slice = Slice.drop_duplicates([Yaxis,Xaxis])

        Slice[Color] = CompressionWeight*Slice[Colors[4]] + TensileWeight*Slice[Colors[7]] + SimpleShearWeight*Slice[Colors[10]]
        Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

        i     = 1
        Pos   = int(i*len(Levels)/(NContours+1))
        ContourSelection = np.zeros(len(Levels))

        while Pos < len(Levels):
            ContourSelection[Pos] = 1
            i  += 1
            Pos = int(i*len(Levels)/(NContours+1))


        SampledLevels = ContourSelection*Levels
        SampledLevels = SampledLevels[SampledLevels != 0]

        SampledLevels = np.insert(SampledLevels, 0, 0)
        SampledLevels = np.append(SampledLevels,Levels[-1])

        Table = Slice.pivot(Yaxis,Xaxis,Color)
        Table = Table.dropna()

        X = Table.columns.values
        Y = Table.index.values
        Z = Table.values
        Z = Z.reshape(Z.size)

        XY,YX = np.meshgrid(X, Y)

        XY = XY.reshape(1,XY.size)[0]
        YX = YX.reshape(1,YX.size)[0]

        # Plot
        ax = fig.add_subplot(1,3,Projection+1)
        ax.scatter(XY, YX, c=Z, marker = '.', s=10, cmap='viridis', vmin=VMin, vmax=VMax)
        ax.tricontour(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, colors='black', linewidths=0.5)
        CostContour = ax.tricontourf(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, cmap='viridis', alpha=0.85)
    #     plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    #     plt.plot(OptiData[Xaxis][len(OptiData)-1], OptiData[Yaxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
        ax.plot([MarcData[Xaxis][len(MarcData)-1]], [MarcData[Yaxis][len(MarcData)-1]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
        if Projection+1 == 3:
            ColorBar=fig.colorbar(CostContour)
            ColorBar.set_label('Cost (-)')
            plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
    #         plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
            plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
            fig.legend(loc='lower center', framealpha=1, ncol=3)
        ax.set_xlabel(Xaxis + Xunits)
        ax.set_ylabel(Yaxis + Yunits)
        plt.subplots_adjust(left=0.05, bottom=0.25, right=0.9, top=0.95, wspace=0.3, hspace=None)
#         plt.savefig('CostFunctionGrid/MarcPoint'+Color+'Results.pdf')
        plt.show()
        fig.canvas.draw_idle()
    
interactive_plot = interactive(f, CompressionWeight=(0.0, 3.0), TensileWeight=(0.0, 3.0), SimpleShearWeight=(0.0, 3.0))
display(interactive_plot)

In [ ]:
%matplotlib widget

# Plot data
Axes  = ['Nu', 'Mu', 'Alpha']
Units = [' (-)', ' (kPa)', ' (-)']
Colors = ['TotalCostNormMax', 'TotalCostNoNorm', 'TotalCostStep', 
          'CompressionCostNoNorm', 'CompressionCostNormMax', 'CompressionCostStep',
          'TensileCostNoNorm', 'TensileCostNormMax', 'TensileCostStep',
          'SimpleShearCostNoNorm', 'SimpleShearCostNormMax', 'SimpleShearCostStep']

Color     = Colors[0]
NContours = 10

Indices = [0,1,2,0,1,2]

plt.rc('figure', figsize=[12,4])
fig = plt.figure()

MinPoint = CostData[Color].idxmin()
    
def f(CompressionWeight, TensileWeight, SimpleShearWeight):
    plt.clf()
    
    for Projection in range(len(Axes)):

        # Plot tuning
        Xaxis     = Axes[Indices[Projection]]
        Xunits    = Units[Indices[Projection]]
        Yaxis     = Axes[Indices[Projection+1]]
        Yunits    = Units[Indices[Projection+1]]
        SliceAxis = Axes[Indices[Projection+2]]

        # Generate data for contour
        Slice4Contour = [CostData[SliceAxis][MinPoint]]
        Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color,Colors[4],Colors[7],Colors[10]]]
        Slice = Slice.drop_duplicates([Yaxis,Xaxis])

        Slice[Color] = CompressionWeight*Slice[Colors[4]] + TensileWeight*Slice[Colors[7]] + SimpleShearWeight*Slice[Colors[10]]
        Levels = Slice.sort_values(by=[Color])[Color].to_numpy()

        i     = 1
        Pos   = int(i*len(Levels)/(NContours+1))
        ContourSelection = np.zeros(len(Levels))

        while Pos < len(Levels):
            ContourSelection[Pos] = 1
            i  += 1
            Pos = int(i*len(Levels)/(NContours+1))


        SampledLevels = ContourSelection*Levels
        SampledLevels = SampledLevels[SampledLevels != 0]

        SampledLevels = np.insert(SampledLevels, 0, 0)
        SampledLevels = np.append(SampledLevels,Levels[-1])

        Table = Slice.pivot(Yaxis,Xaxis,Color)
        Table = Table.dropna()

        X = Table.columns.values
        Y = Table.index.values
        Z = Table.values
        Z = Z.reshape(Z.size)

        XY,YX = np.meshgrid(X, Y)

        XY = XY.reshape(1,XY.size)[0]
        YX = YX.reshape(1,YX.size)[0]

        # Plot
        ax = fig.add_subplot(1,3,Projection+1)
        ax.scatter(XY, YX, c=Z, marker = '.', s=10, cmap='viridis', vmin=VMin, vmax=VMax)
        ax.tricontour(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, colors='black', linewidths=0.5)
        CostContour = ax.tricontourf(XY, YX, Z, vmin=VMin, vmax=VMax, levels=SampledLevels, cmap='viridis', alpha=0.85)
        plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    #     plt.plot(OptiData[Xaxis][len(OptiData)-1], OptiData[Yaxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
#         ax.plot([MarcData[Xaxis][len(MarcData)-1]], [MarcData[Yaxis][len(MarcData)-1]], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
        if Projection+1 == 3:
            ColorBar=fig.colorbar(CostContour)
            ColorBar.set_label('Cost (-)')
            plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
    #         plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData[Color].min()))
            plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCost'][len(OptiData)-1])
            fig.legend(loc='lower center', framealpha=1, ncol=3)
        ax.set_xlabel(Xaxis + Xunits)
        ax.set_ylabel(Yaxis + Yunits)
        plt.subplots_adjust(left=0.05, bottom=0.25, right=0.9, top=0.95, wspace=0.3, hspace=None)
#         plt.savefig('CostFunctionGrid/MarcPoint'+Color+'Results.pdf')
        plt.show()
        fig.canvas.draw_idle()
    
interactive_plot = interactive(f, CompressionWeight=(0.0, 3.0), TensileWeight=(0.0, 3.0), SimpleShearWeight=(0.0, 3.0))
display(interactive_plot)

In [ ]:
Table.values.shape
XY.shape

In [238]:
Figure, Axis = plt.subplots(1, 1, figsize=(10, 7))
Axis.remove()
Axis = Figure.add_subplot(1,1,1, projection='3d')
Xaxis     = Axes[Projection]
Xunits    = Units[Projection]
Yaxis     = Axes[Projection-2]
Yunits    = Units[Projection-2]
SliceAxis = Axes[Projection-1]

# Generate data for contour
Slice4Contour = [CostData[SliceAxis][MinPoint]]
Slice = CostData.loc[CostData[SliceAxis].isin(Slice4Contour)][[Yaxis,Xaxis,Color,Colors[4],Colors[7],Colors[10]]]
Slice = Slice.drop_duplicates([Yaxis,Xaxis])

Table = Slice.pivot(Yaxis,Xaxis,Color)
Table = Table.dropna()

X = Table.columns.values
Y = Table.index.values
Z = Table.values
Z = Z.reshape(Z.size)

XY,YX = np.meshgrid(X, Y)

XY = XY.reshape(1,XY.size)[0]
YX = YX.reshape(1,YX.size)[0]
Axis.tricontour(XY, YX, Z, zdir='z', offset=1, colors='black', linewidths=0.5)
# make the panes transparent
Axis.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
Axis.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
Axis.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# make the grid lines transparent
Axis.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
Axis.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
Axis.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
CostData[SliceAxis][MinPoint]
SliceAxis

In [ ]:
%matplotlib widget

def f(x, y):
    plt.clf()
    plt.plot(x, y, marker='o')
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    fig.canvas.draw_idle()

plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(1,1,1,projection='3D')
interactive_plot = interactive(f, x=(-10.0, 10.0), y=(-10.0, 10.0))
interactive_plot